In [ ]:
import time
import taskingai
# Load TaskingAI API Key from environment variable

# TaskingAI: Chat with Assistant Example

In this example, we will first create an assistant who knows the meaning of various numbers and will explain it in certain language.
Then we will start a chat with the assistant.

##  Create Assistant

In [ ]:
from taskingai.assistant import Assistant, Chat, ToolRef, ToolType
from taskingai.assistant.memory import AssistantMessageWindowMemory

# choose an available chat_completion model from your project
model_id = "YOUR_CHAT_COMPLETION_MODEL_ID"

assistant: Assistant = taskingai.assistant.create_assistant(
    model_id=model_id,
    name="My Assistant",
    description="A assistant who knows the meaning of various numbers.",
    memory=AssistantMessageWindowMemory(
        max_tokens=1000
    ),
    system_prompt_template=[
        "You know the meaning of various numbers.",
        "No matter what the user's language is, you will use the {{langugae}} to explain."
    ],
    tools=[
        ToolRef(
            type=ToolType.PLUGIN,
            id="open_weather/get_hourly_forecast",
        )
    ],
    retrievals=[],
    metadata={"k": "v"},
)
print(f"created assistant: {assistant}\n")

## Start a Chat 

In [ ]:
chat: Chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)
print(f"created chat: {chat.chat_id}\n")

In [ ]:
from taskingai.assistant import Message, MessageChunk
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")

    # generate assistant response
    assistant_message: Message = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        }
    )
    print(f"Assistant: {assistant_message.content.text}")
    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

In [ ]:
user_input = input("User Input: ")
while user_input.strip() and user_input != "q":
    # create user message
    user_message = taskingai.assistant.create_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input} ({user_message.message_id})")

    # generate assistant response
    assistant_message_response = taskingai.assistant.generate_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        },
        stream=True,
    )

    print("Assistant:", end=" ", flush=True)
    for item in assistant_message_response:
        if isinstance(item, MessageChunk):
            print(item.delta, end="", flush=True)
        elif isinstance(item, Message):
            print(f" ({item.message_id})")

    time.sleep(2)
    # quit by input 'q
    user_input = input("User: ")

In [ ]:
# clean chat context
taskingai.assistant.clean_chat_context(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)

In [ ]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
    order="asc"
)
for message in messages:
    print(f"{message.role}: {message.content.text}")

In [ ]:
# delete assistant
taskingai.assistant.delete_assistant(
    assistant_id=assistant.assistant_id,
)